In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn
import torch
RANDOM_SEED = 42

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

NUM_CLASSES = 4

In [34]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED, shuffle=True)

In [35]:
X_train, y_train, X_test, y_test = map(torch.tensor, (X_train, y_train, X_test, y_test))

In [36]:
X_train = X_train.float()
y_train = y_train.long()

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 


X_train=X_train.to(device)
y_train=y_train.to(device)
X_test=X_test.to(device)
y_test=y_test.to(device)

In [38]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


torch.Size([3590, 42]) torch.Size([3590])
torch.Size([1197, 42]) torch.Size([1197])


# Model building

In [39]:

class FeedForward(nn.Module): # Almost everything in PyTorch inherits from nn.Module

  def __init__(self, input_shape: int, hidden_units: int, output_shape: int ):
    super().__init__()
    self.block_1 = nn.Sequential(
      nn.Dropout(0.20),
      nn.Linear(in_features = input_shape,
                out_features = hidden_units
                                ),
      nn.ReLU(),
    )

    self.block_2 = nn.Sequential(
                                  nn.Dropout(0.40),
                                  nn.Linear(in_features = hidden_units,
                                            out_features = hidden_units
                                                              ),
                                  nn.ReLU(),
                                  nn.Linear(in_features = hidden_units,
                                            out_features = output_shape
                                                              ),
    )

  
    

# Forward method to define computation in the model
  def forward(self, x: torch.Tensor)->torch.Tensor: # "x" is the imput data
    return  self.block_2(self.block_1(x))


  def predict(self, X):
    Y_pred = self.forward(X)
    return Y_pred
torch.manual_seed(RANDOM_SEED)
model_1 = FeedForward(input_shape=42, hidden_units=40, output_shape=NUM_CLASSES).to(device)

In [40]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_1.parameters(), lr = 0.01)  

In [41]:
def fit_v2(x, y, model, opt, loss_fn, epochs = 1000):
  
  for epoch in range(epochs):
    loss = loss_fn(model(x), y)
    loss.backward()
    opt.step()
    opt.zero_grad()
    
  return loss.item()

fit_v2(X_train, y_train, model_1, optimizer, loss_fn)

0.15877516567707062

In [42]:
def accuracy(y_hat, y):
  pred = torch.argmax(y_hat, dim=1)
  return (pred == y).float().mean()

In [43]:
Y_pred_train = model_1.predict(X_train)
#Y_pred_train = np.argmax(Y_pred_train,1)
Y_pred_val = model_1.predict(X_test)
#Y_pred_val = np.argmax(Y_pred_val,1)

accuracy_train = accuracy(Y_pred_train, y_train)
accuracy_val = accuracy(Y_pred_val, y_test)

In [44]:
print(accuracy_train)
print(accuracy_val)

tensor(0.9446)
tensor(0.9524)
